In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
import pandas as pd
import bitsandbytes
from transformers import TrainingArguments
from transformers import Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig,get_peft_model

In [2]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load the tokenizer
MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B"


In [4]:
# Explicitly setting the tokenizer type
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    trust_remote_code=True,  # Allow loading custom tokenizer implementations
)

In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [6]:
# Load the model with 8-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_4bit=True  # Enable 8-bit quantization
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [7]:
lora_config = LoraConfig(
    r=16,                # rank of LoRA updates
    lora_alpha=32,       # scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # adjust target modules as needed
    lora_dropout=0.1,    # dropout for LoRA layers
    bias="none",
    task_type="CAUSAL_LM"
)

In [8]:
# Wrap the model with PEFT/LoRA
model = get_peft_model(model, lora_config)
print("LoRA model parameters:")
model.print_trainable_parameters()

LoRA model parameters:
trainable params: 4,358,144 || all params: 1,781,446,144 || trainable%: 0.2446


In [9]:
DATA_FILE = 'D:/Projects/Chat Assistant for SQLite Database/FineTune/SQL_CustumDataSet/spider_text_sql.csv'
MAX_SEQ_LENGTH = 1024

In [10]:
def format_example(example):
    # Concatenate the 'intput' and 'output' columns with a newline and EOS token.
    text = example["text_query"].strip() + "\n" + example["sql_command"].strip() + tokenizer.eos_token
    return {"text": text}

# Load the CSV dataset.
dataset = load_dataset("csv", data_files={"train": DATA_FILE})
# Map each example to a unified text field.
dataset = dataset["train"].map(format_example)

In [11]:
# Tokenize the dataset.
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding="max_length"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

In [12]:
# Create a data collator for language modeling.
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [13]:
training_args = TrainingArguments(
    output_dir="./deepseek_r1_finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # simulate a larger batch size
    num_train_epochs=3,
    learning_rate=2e-2,
    logging_steps=5,
    save_steps=50,
    fp16=True,  # mixed precision if supported
    report_to="none",
)

In [ ]:
#Google Colab
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deepseek_r1_finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # Simulates a larger batch size
    num_train_epochs=3,
    learning_rate=2e-5,  # Reduced learning rate for stability
    logging_steps=10,  # Log every 10 steps
    save_steps=100,  # Save checkpoint every 100 steps
    fp16=True,  # Enable mixed precision training
    report_to="none",
)


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,)

In [15]:
print("Starting training...")
trainer.train()
print("Training complete.")

Starting training...


c:\Users\rajpu\anaconda3\envs\CudaEnv\Lib\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
5,17.300000
10,17.346100
15,157.042100
20,160577.262500
25,803.355500
30,10273.712500
35,0.000000


KeyboardInterrupt: 

In [ ]:
# Save Model and Tokenizer
output_dir = 'fine_tuned_deepseek'
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)